In [24]:
import datasets
from t5_tokenizer_model import SentencePieceUnigramTokenizer

vocab_size = 32000

# size of the input data
input_sentence_size = None

cache_dir="/workdir/.cache/huggingface/datasets"

# using a tiny bit of the dataset to train a tokenizer
ds = datasets.load_dataset("oscar",  
    name="unshuffled_deduplicated_no", 
    cache_dir=cache_dir, 
    split="train[:100]"
    )

len(ds)

Reusing dataset oscar (/workdir/.cache/huggingface/datasets/oscar/unshuffled_deduplicated_no/1.0.0/84838bd49d2295f62008383b05620571535451d84545037bb94d6f3501651df2)


100

In [47]:
import os
tokenizer = SentencePieceUnigramTokenizer(unk_token="<unk>", eos_token="</s>", pad_token="<pad>")

def batch_iterator(ds, input_sentence_size=None):
    if input_sentence_size==None:
        input_sentence_size = len(ds)
    batch_size = 100
    for i in range(0, input_sentence_size, batch_size):
        yield ds[i: i+batch_size]["text"]

tokenizer.train_from_iterator(
    iterator=batch_iterator(ds),
    vocab_size=vocab_size,
    show_progress=True
)

t5_config_dir = "/workdir/norwegian-t5-base/"
if not os.path.exists(t5_config_dir):
    os.makedirs(t5_config_dir)

tokenizer.save(os.path.join(t5_config_dir, "tokenizer.json"))

In [48]:
tokenizer.get_vocab_size()

6152

In [33]:

from transformers import T5Config

config = T5Config.from_pretrained("google/t5-v1_1-base", vocab_size=tokenizer.get_vocab_size())
config.save_pretrained(t5_config_dir)

In [46]:
from transformers import AutoTokenizer
from transformers import T5Config
# use rust based tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "/workdir/norwegian-t5-base", 
    cache_dir="/workdir/norwegian-t5-base",
    #use_fast=True,
    use_auth_token=None
)




Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file /workdir/norwegian-t5-base/config.json
Model config T5Config {
  "_name_or_path": "/workdir/norwegian-t5-base",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 768,
  "decoder_start_token_id": 0,
  "dense_act_fn": "gelu_new",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "gated-gelu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": true,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "num_decoder_layers": 12,
  "num_heads": 12,
  "num_layers": 12,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "tie_word_embeddings": false,
  "transformers_version": "4.22.0.dev0",
  "use_cache": true,
  "vocab_size": 6152
}

loading file spiece.model
loading file tokenizer.json
loading file added

In [ ]:
config = T5Config.from_pretrained(
    "/workdir/norwegian-t5-base/",
    cache_dir=cache_dir,
    vocab_size = len(tokenizer),
)